In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
import psutil

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model

from keras import optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras

Using TensorFlow backend.


In [2]:
print(os.listdir('E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'))

['embedding_matrix1.npy', 'testfinal.csv', 'testready.csv', 'testtemp.csv', 'trainfinal.csv', 'trainready.csv', 'traintemp.csv']


In [3]:
def get_keras_data(data1,data2,data3):
    title=[]
    for i in list(data1.title):
        title.append(list(eval(i)))
    description=[]
    for i in list(data1.description):
        description.append(list(eval(i)))
    
    title=pad_sequences(title, maxlen=16)
    description=pad_sequences(description, maxlen=128)
    parent_category_name=np.array(data2.parent_category_name)
    category_name=np.array(data2.category_name)
    param_1=np.array(data2.param_1)
    param123=np.array(data2.param123)
    image_top_1=np.array(data2.image_top_1)
    cluster_5=np.array(data2.cluster_5)
    cluster_10=np.array(data2.cluster_10)
    cluster_20=np.array(data2.cluster_20)
    user_type=np.array(data2.user_type)
    activation_date=np.array(data2.activation_date)
    item_seq=np.array(data2.item_seq_number)
    
    columns=['item_seq_count','item_seq_deal_avg','dis_days', 'edit_time', 'item_num','count']
    numerical=np.array(data3[columns])
    columns=['avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
             'avg_pixel_width', 'width', 'height', 'size',
             'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2',
             'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2',
             'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2']
    image=np.array(data3[columns])
    columns=['cus', 'ship', 'r_cus', 'r_ship', 'all_cus']
    location=np.array(data3[columns])
    price=np.array(data3.price)
    columns=['para_num',
             'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
             'title_num_words','title_num_unique_words','title_words_vs_unique']
    prenum=np.array(data3[columns])
    columns=['description_num_words','description_num_unique_words','description_words_vs_unique']
    postnum=np.array(data3[columns])
    
    
    print("Data ready for Vectorization")
    
    X=[title, description, parent_category_name, category_name, param_1, param123, image_top_1,
       cluster_5, cluster_10, cluster_20, user_type, activation_date, item_seq,
       numerical,image,location,price,prenum,postnum]
    
    return X

In [4]:
columns1=['item_id','title', 'description']
columns2=['parent_category_name','category_name', 'user_type','activation_date','item_seq_number',
         'param_1', 'param123','image_top_1','cluster_5', 'cluster_10','cluster_20']
columns3=['item_seq_count','item_seq_deal_avg','dis_days', 'edit_time', 'item_num','count',
          'avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
          'avg_pixel_width', 'width', 'height', 'size',
          'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2',
          'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2',
          'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2',
          'cus', 'ship', 'r_cus', 'r_ship', 'all_cus',
          'para_num',
          'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
          'title_num_words','title_num_unique_words','title_words_vs_unique',
          'description_num_words','description_num_unique_words','description_words_vs_unique',
          'price']
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
data1=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns1)
data2=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns2)
data3=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns3)

X_test= get_keras_data(data1,data2,data3)
del data2,data3

gc.collect()

Data ready for Vectorization


14

In [5]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred))) 
modelRNN=keras.models.load_model('bestwhole.hdf5',custom_objects={'root_mean_squared_error': root_mean_squared_error})


In [6]:
Y_preds,title,description = modelRNN.predict(X_test,verbose=1)
Y_preds=Y_preds.flatten()
Y_preds

1503424/1503424 [==============================] - 3475s 2ms/step


array([0.09924212, 0.58702374, 0.04933553, ..., 0.39263773, 0.02905533,
       0.11105549], dtype=float32)

In [7]:
result = {'item_id':data1.item_id, 'deal_probability': Y_preds}
result = pd.DataFrame(data=result)
result=result[['item_id','deal_probability']]
result.to_csv('resultRNNtrain.csv', index=False) 

In [8]:
temp1=np.asmatrix(title)
temp1=pd.DataFrame(temp1)
temp1['item_id'] = pd.Series(data1.item_id, index=temp1.index)
temp2=np.asmatrix(description)
temp2=pd.DataFrame(temp2)
temp2['item_id'] = pd.Series(data1.item_id, index=temp1.index)

data=pd.merge(temp1,temp2, on=['item_id'],how='left')
data.to_csv('RNNtrain.csv', index=False)

data


,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y
0,0.693724,1.330172,0.0,0.949546,0.364945,0.296114,0.748295,0.000000,0.000000,0.652646,...,0.763977,0.000000,0.0,0.000000,0.491658,0.000000,0.000000,0.134477,0.000000,0.0
1,0.362372,0.000000,0.0,0.783657,0.371156,0.988188,0.000000,0.000000,0.000000,0.080645,...,0.838433,0.000000,0.0,0.000000,0.556843,0.000000,0.000000,0.223157,0.000000,0.0
2,0.983348,1.342826,0.0,1.116797,0.277078,0.352688,0.609253,0.000000,0.114366,0.606624,...,0.475651,0.000000,0.0,0.000000,0.243584,0.000000,0.000000,0.195716,0.000000,0.0
3,0.661531,1.357864,0.0,1.034240,0.513638,0.000000,0.955351,0.000000,0.509102,0.674689,...,0.378359,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.020177,0.000000,0.0
4,0.949552,1.341449,0.0,1.412675,0.544888,0.253057,0.576460,0.000000,0.402002,0.977763,...,0.315139,0.000000,0.0,0.000000,0.164881,0.000000,0.000000,0.297465,0.000000,0.0
5,0.817023,1.224058,0.0,1.133170,0.249338,0.409103,0.766963,0.000000,0.176517,0.547344,...,0.265149,0.000000,0.0,0.000000,0.409623,0.000000,0.000000,0.066424,0.000000,0.0
6,0.534099,0.693796,0.0,0.797196,0.623332,0.251140,0.842396,0.000000,0.281144,0.369672,...,0.325132,0.000000,0.0,0.000000,0.203015,0.000000,0.102743,0.301351,0.453779,0.0
7,0.000000,0.189241,0.0,0.516867,0.128758,0.000000,0.653707,0.196213,0.340285,0.640264,...,0.305898,0.063186,0.0,0.022176,0.307587,0.053471,0.000000,0.275332,0.000000,0.0
8,0.882145,1.676187,0.0,1.339163,0.661126,0.256402,0.767099,0.000000,0.342429,0.597667,...,0.440981,0.000000,0.0,0.000000,0.388069,0.000000,0.000000,0.104921,0.000000,0.0
9,0.000000,0.056637,0.0,0.520007,0.000000,0.000000,1.554753,0.000000,0.212591,1.106595,...,0.355369,0.062383,0.0,0.205679,0.379737,0.000000,0.000000,0.305272,0.000000,0.0
